In [12]:
import pandas as pd
import json
import os
import dotenv
from pymongo import MongoClient
import pymongo



In [3]:
# I create the connection with the database called 'companies'.
client = MongoClient("localhost:27017")
db = client['geospatial']
companies = db.get_collection('companies')


In [4]:
# I set the projection and the filters to query the mongoDB
projection = {}
filter_ = {'offices.country_code':'AUS'}


In [5]:
# I perform the request
result = list(companies.find(filter_,projection))
print(f'I got {len(result)} companies in AUS')


I got 257 companies in AUS


In [6]:
# I create a dictionary only with the information I really need from each company, in this case, the offices.
dict_of_companies = {}
for i in result:
    dict_of_companies[i['name']]= i['offices']


In [7]:
# I create a list of dictionaries, each of them contains info about the company and all the offices in aus for each company. 

list_offices_aus = []
# I loop through the companies' names
for company in dict_of_companies.keys():

    # I create a list containing the offices information, so each element is an office.
    list_of_offices = dict_of_companies[company]
    for office in list_of_offices:

        # for each office, I get the information that I need, based on a condition, and I create a dictionary formated correctly.
        if office['country_code'] == 'AUS':
            list_offices_aus.append({'company':company,
                                'name_office':office['description'],
                                'city': office['city'],
                                'address': office['address1'],
                                'location':{'type':'Point',
                                            'coordinates':[office['latitude'],office['longitude']]},
                                'latitude':office['latitude'],
                                'longitude':office['longitude']
                                            })
                                

In [8]:
# I upload the dictionary to mongoDB
aus_offices = db['aus_offices']
aus_offices.insert_many(list_offices_aus)